In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import os 
from scipy.stats import wilcoxon, ttest_rel
from tabulate import tabulate

from slim_gsgp_lib.datasets.data_loader import *
datasets = [globals()[i] for i in globals() if 'load' in i][2:]
datasets = datasets[:12] + datasets[13:]  # EXCLUDE PARKINSONS

dataset_dict = {}
df_datasets = {}
for i, dataset in enumerate(datasets):
    X,y = dataset()
    name = dataset.__name__.split('load_')[1]
    # id should be a two digit number
    id = 'DA' + str(i).zfill(2)
    dataset_dict[name] = id 
    df_datasets[name] = X.shape[0], X.shape[1]

df_datasets

{'resid_build_sale_price': (372, 107),
 'istanbul': (536, 7),
 'airfoil': (1503, 5),
 'bike_sharing': (731, 13),
 'boston': (506, 13),
 'breast_cancer': (569, 30),
 'concrete_slump': (103, 7),
 'concrete_strength': (1005, 8),
 'diabetes': (442, 10),
 'efficiency_heating': (768, 8),
 'efficiency_cooling': (768, 8),
 'forest_fires': (513, 43),
 'ld50': (234, 626),
 'ppb': (131, 626),
 'bioav': (358, 241)}

In [115]:
def get_significance(p_value, ratio):
    if p_value >= 0.05:
        return 'NSD'
    elif ratio > 1:
        return '-' * (1 + int(p_value < 0.01) + int(p_value < 0.001))
    else:
        return '+' * (1 + int(p_value < 0.01) + int(p_value < 0.001))

def means_df(prefixes=['sc', 'scsm'],
             datasets=None,
             best=False,
             rmse_compare=True,
             decimals=3,
             table=False):
    
    if datasets is None:
        datasets = dataset_dict.keys()
    
    rows_rmse, rows_rmse_std, rows_size, rows_size_std = [], [], [], []
    rmse_sig_test, size_sig_test = {}, {}

    for dataset in datasets:
        if dataset not in rmse_sig_test:
            rmse_sig_test[dataset] = {}
            size_sig_test[dataset] = {}

        for prefix in prefixes: 
            if prefix not in rmse_sig_test[dataset]:
                rmse_sig_test[dataset][prefix] = {}
                size_sig_test[dataset][prefix] = {}
                
            with open(f'results/slim/{dataset_dict[dataset]}/{prefix}.pkl', 'rb') as f:
                results = pickle.load(f)
                rmse = results['rmse_compare'] if rmse_compare else results['rmse']
                size = results['size']

            for algo, values in rmse.items():

                rows_rmse.append({'Dataset': dataset, 'Algorithm': algo, f'rmse_{prefix}': np.mean(values)})
                rows_size.append({'Dataset': dataset, 'Algorithm': algo, f'size_{prefix}': np.mean(size[algo])})
                rows_rmse_std.append({'Dataset': dataset, 'Algorithm': algo, f'rmse_std_{prefix}': np.std(values)})
                rows_size_std.append({'Dataset': dataset, 'Algorithm': algo, f'size_std_{prefix}': np.std(size[algo])})
                rmse_sig_test[dataset][prefix][algo] = np.array(values)
                size_sig_test[dataset][prefix][algo] = np.array(size[algo])
                
    df_rmse = pd.DataFrame(rows_rmse)
    df_size = pd.DataFrame(rows_size)
    df_rmse_std = pd.DataFrame(rows_rmse_std)
    df_size_std = pd.DataFrame(rows_size_std)

    # Pivot tables and explicitly reorder columns based on the given prefixes
    df_rmse = df_rmse.pivot_table(index=['Dataset', 'Algorithm'], values=[f'rmse_{p}' for p in prefixes], 
                                aggfunc='first').reset_index()
    df_size = df_size.pivot_table(index=['Dataset', 'Algorithm'], values=[f'size_{p}' for p in prefixes], 
                                aggfunc='first').reset_index()
    df_rmse_std = df_rmse_std.pivot_table(index=['Dataset', 'Algorithm'], values=[f'rmse_std_{p}' for p in prefixes],
                                        aggfunc='first').reset_index()
    df_size_std = df_size_std.pivot_table(index=['Dataset', 'Algorithm'], values=[f'size_std_{p}' for p in prefixes],
                                        aggfunc='first').reset_index()
    
    # Explicitly reorder columns based on the prefixes
    df_rmse = df_rmse[['Dataset', 'Algorithm'] + [f'rmse_{p}' for p in prefixes]]
    df_size = df_size[['Dataset', 'Algorithm'] + [f'size_{p}' for p in prefixes]]
    df_rmse_std = df_rmse_std[['Dataset', 'Algorithm'] + [f'rmse_std_{p}' for p in prefixes]]
    df_size_std = df_size_std[['Dataset', 'Algorithm'] + [f'size_std_{p}' for p in prefixes]]
    df_rmse['ratio_rmse'] = df_rmse[f'rmse_{prefixes[0]}'] / df_rmse[f'rmse_{prefixes[1]}']
    df_size['ratio_size'] = df_size[f'size_{prefixes[0]}'] / df_size[f'size_{prefixes[1]}']

    
    # Calculate the statistcial significance
    for dataset in rmse_sig_test.keys():
        for algo in rmse_sig_test[dataset][prefixes[0]].keys():
            for prefix in prefixes:
                if prefix == prefixes[0]:
                    continue
                if len(rmse_sig_test[dataset][prefix][algo]) != len(rmse_sig_test[dataset][prefixes[0]][algo]):
                    min_len = min(len(rmse_sig_test[dataset][prefix][algo]), len(rmse_sig_test[dataset][prefixes[0]][algo]))
                    rmse_sig_test[dataset][prefix][algo] = rmse_sig_test[dataset][prefix][algo][:min_len]
                    rmse_sig_test[dataset][prefixes[0]][algo] = rmse_sig_test[dataset][prefixes[0]][algo][:min_len]
                    size_sig_test[dataset][prefix][algo] = size_sig_test[dataset][prefix][algo][:min_len]
                    size_sig_test[dataset][prefixes[0]][algo] = size_sig_test[dataset][prefixes[0]][algo][:min_len]

                _, p_value_rmse = wilcoxon(np.round(rmse_sig_test[dataset][prefix][algo] - rmse_sig_test[dataset][prefixes[0]][algo], decimals=8), zero_method='pratt', alternative='two-sided', method='approx')
                _, p_value_size = wilcoxon(np.round(size_sig_test[dataset][prefix][algo] - size_sig_test[dataset][prefixes[0]][algo], decimals=8), zero_method='pratt', alternative='two-sided', method='approx')

                df_rmse.loc[(df_rmse['Dataset'] == dataset) & (df_rmse['Algorithm'] == algo), 'rmse_significance'] = \
                    get_significance(
                        p_value_rmse, 
                        df_rmse.loc[(df_rmse['Dataset'] == dataset) & (df_rmse['Algorithm'] == algo), f'ratio_rmse'].values[0]
                    )

                df_size.loc[(df_size['Dataset'] == dataset) & (df_size['Algorithm'] == algo), 'size_significance'] = \
                    get_significance(
                        p_value_size, 
                        df_size.loc[(df_size['Dataset'] == dataset) & (df_size['Algorithm'] == algo), f'ratio_size'].values[0]
                    )
                
    df_rmse.set_index(['Dataset', 'Algorithm'], inplace=True)
    df_size.set_index(['Dataset', 'Algorithm'], inplace=True)
    df_rmse_std.set_index(['Dataset', 'Algorithm'], inplace=True)
    df_size_std.set_index(['Dataset', 'Algorithm'], inplace=True)


    if best:
        return _means_df_best(df_rmse, df_size, df_rmse_std, df_size_std, prefixes, rmse_compare, decimals, table)
    
    elif table:
        df_combined = pd.concat([df_rmse, df_rmse_std, df_size, df_size_std], axis=1).reset_index()
        df_combined = df_combined.round(decimals)
        for prefix in prefixes:
            df_combined[f'rmse_{prefix}'] = df_combined[f'rmse_{prefix}'].astype(str) + ' ± ' + df_combined[f'rmse_std_{prefix}'].astype(str)
            df_combined[f'size_{prefix}'] = df_combined[f'size_{prefix}'].astype(str) + ' ± ' + df_combined[f'size_std_{prefix}'].astype(str)
        df_combined = df_combined[[col for col in df_combined.columns if '_std' not in col]]
        reordered_columns = (
            ['Dataset', 'Algorithm'] +
            [f'rmse_{p}' for p in prefixes] +
            [f'size_{p}' for p in prefixes] +
            ['ratio_rmse', 'ratio_size'] + 
            ['rmse_significance', 'size_significance']
        )
        df_combined = df_combined[reordered_columns]
        print(tabulate(df_combined, headers='keys', tablefmt='fancy_grid', floatfmt=f".{decimals}f"))
        return None
    
    # Combine df_rmse and df_size
    df_combined = pd.concat([df_rmse, df_size], axis=1).reset_index()
    df_combined = df_combined.round(decimals)
    reordered_columns = (
        ['Dataset', 'Algorithm'] +
        [f'rmse_{p}' for p in prefixes] +
        [f'size_{p}' for p in prefixes] +
        ['ratio_rmse', 'ratio_size'] + 
        ['rmse_significance', 'size_significance']
    )
    df_combined = df_combined[reordered_columns]
    df_combined.set_index(['Dataset', 'Algorithm'], inplace=True)
    return df_combined.sort_values('ratio_rmse')    


def _means_df_best(df_rmse, 
                   df_size, 
                   df_rmse_std,
                   df_size_std,
                   prefixes,
                   rmse_compare=True,
                   decimals=3, 
                   table=False):
    best_dict_rmse, best_dict_size, best_dict_rmse_std, best_dict_size_std = {}, {}, {}, {}
    prefix_columns = df_rmse.columns[:-1]

    rmse_sig_test, size_sig_test = {}, {}
    for prefix in prefix_columns:
        best_indices = df_rmse.groupby('Dataset')[prefix].idxmin().values

        for idx in best_indices:
            dataset, algo = idx

            # Significance test
            dataset_id = dataset_dict[dataset]
            with open(f'results/slim/{dataset_id}/{prefix.replace("rmse_", "")}.pkl', 'rb') as f:
                results = pickle.load(f)
                rmse = results['rmse_compare'][algo] if rmse_compare else results['rmse'][algo]
                size = results['size'][algo]
            if dataset not in rmse_sig_test:
                rmse_sig_test[dataset] = {}
                size_sig_test[dataset] = {}
            rmse_sig_test[dataset][prefix] = np.array(rmse)
            size_sig_test[dataset][prefix] = np.array(size)

            # Append to best_dict
            rmse_value = df_rmse.loc[idx, prefix]
            size_value = df_size.loc[idx, prefix.replace('rmse', 'size')]
            rmse_value_std = df_rmse_std.loc[idx, f'rmse_std_{prefix.replace("rmse_", "")}']
            size_value_std = df_size_std.loc[idx, f'size_std_{prefix.replace("rmse_", "")}']

            if dataset not in best_dict_rmse:
                best_dict_rmse[dataset] = []
                best_dict_size[dataset] = []
                best_dict_rmse_std[dataset] = []
                best_dict_size_std[dataset] = []
            best_dict_rmse[dataset].append(rmse_value)   
            best_dict_size[dataset].append(size_value)
            best_dict_rmse_std[dataset].append(rmse_value_std)
            best_dict_size_std[dataset].append(size_value_std)

    best_df_rmse = pd.DataFrame(best_dict_rmse).T
    best_df_size = pd.DataFrame(best_dict_size).T
    best_df_rmse_std = pd.DataFrame(best_dict_rmse_std).T
    best_df_size_std = pd.DataFrame(best_dict_size_std).T
    best_df_rmse.columns = prefix_columns
    best_df_size.columns = [c.replace('rmse', 'size') for c in prefix_columns]
    best_df_rmse_std.columns = [f'{c}_std' for c in prefix_columns]
    best_df_size_std.columns = [f'{c}_std' for c in best_df_size.columns]
    best_df_rmse['ratio_rmse'] = best_df_rmse[prefix_columns[0]] / best_df_rmse[prefix_columns[1]]
    best_df_size['ratio_size'] = best_df_size[prefix_columns[0].replace('rmse', 'size')] / best_df_size[prefix_columns[1].replace('rmse', 'size')]

    # For each dataset, calculate the significance test
    for dataset in rmse_sig_test.keys():
        for prefix in prefix_columns:
            if prefix == prefix_columns[0]:
                continue
            # Check if the lengths are the same
            if len(rmse_sig_test[dataset][prefix]) != len(rmse_sig_test[dataset][prefix_columns[0]]):
                min_len = min(len(rmse_sig_test[dataset][prefix]), len(rmse_sig_test[dataset][prefix_columns[0]]))
                rmse_sig_test[dataset][prefix] = rmse_sig_test[dataset][prefix][:min_len]
                rmse_sig_test[dataset][prefix_columns[0]] = rmse_sig_test[dataset][prefix_columns[0]][:min_len]
                size_sig_test[dataset][prefix] = size_sig_test[dataset][prefix][:min_len]
                size_sig_test[dataset][prefix_columns[0]] = size_sig_test[dataset][prefix_columns[0]][:min_len]
                
            _, p_value_rmse = wilcoxon(np.round(rmse_sig_test[dataset][prefix] - rmse_sig_test[dataset][prefix_columns[0]], decimals=8), zero_method='pratt', alternative='two-sided', method='approx')
            _, p_value_size = wilcoxon(np.round(size_sig_test[dataset][prefix] - size_sig_test[dataset][prefix_columns[0]], decimals=8), zero_method='pratt', alternative='two-sided', method='approx')
            best_df_rmse.loc[dataset, 'rmse_significance'] = get_significance(p_value_rmse, best_df_rmse.loc[dataset, 'ratio_rmse'])
            best_df_size.loc[dataset, 'size_significance'] = get_significance(p_value_size, best_df_size.loc[dataset, 'ratio_size'])

    best_df_rmse.sort_values('ratio_rmse')
    best_df = pd.concat([best_df_rmse, best_df_size], axis=1)
    best_df = best_df.round(decimals)

    # If table, we want to add the standard deviations in the rmses and sizes
    if table:
        best_df_std = pd.concat([best_df_rmse, best_df_size, best_df_rmse_std, best_df_size_std], axis=1)
        best_df_std = best_df_std.round(decimals)
        best_df_std = best_df_std.sort_values('ratio_rmse')
        
        # I need to have the rmse_scsm +- rmse_scsm_std and the same for size
        best_df_std[f'rmse_{prefixes[0]}'] = best_df_std[f'rmse_{prefixes[0]}'].astype(str) + ' ± ' + best_df_std[f'rmse_{prefixes[0]}_std'].astype(str)
        best_df_std[f'rmse_{prefixes[1]}'] = best_df_std[f'rmse_{prefixes[1]}'].astype(str) + ' ± ' + best_df_std[f'rmse_{prefixes[1]}_std'].astype(str)
        best_df_std[f'size_{prefixes[0]}'] = best_df_std[f'size_{prefixes[0]}'].astype(str) + ' ± ' + best_df_std[f'size_{prefixes[0]}_std'].astype(str)
        best_df_std[f'size_{prefixes[1]}'] = best_df_std[f'size_{prefixes[1]}'].astype(str) + ' ± ' + best_df_std[f'size_{prefixes[1]}_std'].astype(str)

        # Drop columns that contain _std
        best_df_std = best_df_std[[col for col in best_df_std.columns if '_std' not in col]]

        # Now print the table
        print(tabulate(best_df_std, headers='keys', tablefmt='fancy_grid', floatfmt=f".{decimals}f"))
        return None

    return best_df.sort_values('ratio_rmse')

In [119]:
means_df(prefixes=['scsm', 'sc'],
         best=False, rmse_compare=False, decimals=4, table=False).value_counts('rmse_significance')

rmse_significance
NSD    48
+++    15
+      13
++     11
-       1
--      1
---     1
Name: count, dtype: int64

In [23]:
df = means_df(prefixes=['scsm', 'sc'],
         best=False, rmse_compare=False, decimals=4, table=False)

In [49]:
df = means_df(prefixes=['scsm', 'sc'],
         best=False, rmse_compare=False, decimals=4, table=False)

# For each dataset, apply a scaler to the rmse_scsm and rmse_sc columns
maxs = df.groupby('Dataset').max()[['rmse_scsm', 'rmse_sc']].max(axis=1)
mins = df.groupby('Dataset').min()[['rmse_scsm', 'rmse_sc']].min(axis=1)
df['rmse_scsm_scaled'] = (df['rmse_scsm'] - mins) / (maxs - mins)
df['rmse_sc_scaled'] = (df['rmse_sc'] - mins) / (maxs - mins)

# Scale the sizes
maxs = df.groupby('Dataset').max()[['size_scsm', 'size_sc']].max(axis=1)
mins = df.groupby('Dataset').min()[['size_scsm', 'size_sc']].min(axis=1)
df['size_scsm_scaled'] = (df['size_scsm'] - mins) / (maxs - mins)
df['size_sc_scaled'] = (df['size_sc'] - mins) / (maxs - mins)

# Create a score column
df['score_scsm'] = df['rmse_scsm_scaled'] + 0.5 * df['size_scsm_scaled']
df['score_sc'] = df['rmse_sc_scaled'] + 0.5 * df['size_sc_scaled']

# Drop the scaled columns
df.drop(columns=['rmse_scsm_scaled', 'rmse_sc_scaled', 'size_scsm_scaled', 'size_sc_scaled'], inplace=True)

In [56]:
# Get, for each dataset the best algorithm
best_indices = df.groupby('Dataset')[['score_scsm', 'score_sc']].idxmin().values.flatten()
best_df = df.loc[best_indices]

# For which datasets is scsm better than sc? Youll need to pivot the table
df

rmse_scsm  rmse_sc  size_scsm   size_sc  \
Dataset            Algorithm                                            
bike_sharing       SUM_SIG2      0.0269   0.0580    85.7000  150.0000   
                   MUL_ABS       0.0341   0.0627    86.8333   77.1000   
                   MUL_SIG2      0.0225   0.0405    87.0333   80.3000   
                   MUL_SIG1      0.0299   0.0528    90.0667  150.0333   
ppb                MUL_SIG1      0.2841   0.4905    91.8000  181.4333   
...                                 ...      ...        ...       ...   
istanbul           MUL_SIG1      0.0738   0.0633    91.7000  148.9667   
efficiency_cooling SUM_ABS       0.0847   0.0705    95.1000  278.6667   
ppb                SUM_ABS       0.3847   0.2692    87.3667   89.0667   
                   SUM_SIG1      0.4478   0.2730    89.7000  172.9000   
                   MUL_ABS       0.4868   0.2816    99.0000   87.2667   

                              ratio_rmse  ratio_size  score_scsm  score_sc  \
Dataset            Algorithm                                                 
bike_sharing       SUM_SIG2       0.4628      0.5713    0.167079  1.226552   
                   MUL_ABS        0.5429      1.1262    0.347993  1.000000   
                   MUL_SIG2       0.5555      1.0839    0.066002  0.473513   
                   MUL_SIG1       0.5663      0.6003    0.260687  1.099878   
ppb                MUL_SIG1       0.5793      0.5060    0.119997  1.500000   
...                                  ...         ...         ...       ...   
istanbul           MUL_SIG1       1.1668      0.6156    1.052737  0.497525   
efficiency_cooling SUM_ABS        1.2018      0.3413    0.531005  0.500000   
ppb                SUM_ABS        1.4288      0.9809    0.537106  0.040219   
                   SUM_SIG1       1.6402      0.5188    0.825886  0.501997   
                   MUL_ABS        1.7288      1.1345    1.046094  0.084976   

                              scsm_better  
Dataset            Algorithm               
bike_sharing       SUM_SIG2          True  
                   MUL_ABS           True  
                   MUL_SIG2          True  
                   MUL_SIG1          True  
ppb                MUL_SIG1          True  
...                                   ...  
istanbul           MUL_SIG1         False  
efficiency_cooling SUM_ABS          False  
ppb                SUM_ABS          False  
                   SUM_SIG1         False  
                   MUL_ABS          False  

[90 rows x 9 columns]

### PARAMS